In [ ]:
#this algorithm extrapolates the wind profile using NBeats
#the hyperparameters were optimized with Optuna v. 4.2.1
#df is the dataset


timesteps = 1
windows, labels = prep_time_series(df[:12000,1], lookback=timesteps, horizon=1)
print(windows)

windows2, labels2 = prep_time_series(df[timesteps:12000,5], lookback=0, horizon=1)
print(labels2)
X_train, X_test, y_train, y_test = train_test_split(windows, labels2,
shuffle = False, test_size = 0.1)

num_generic_layers = 2
num_generic_neurons = 896
batch_size = 664
nbeats = NBeatsModel(num_generic_layers=num_generic_layers,
                       num_generic_neurons = num_generic_neurons,
                       lookback=timesteps, horizon=1, batch_size=batch_size)
nbeats.build_layer()
nbeats.build_model()
time_input = keras.layers.Input(shape = (timesteps,))
nbeats_layer = nbeats.model_layer(time_input)
non_time_input  = keras.layers.Input(shape=(4,))
units = 27
x = keras.layers.Dense(units = units, activation='relu')(non_time_input)
concat = keras.layers.Concatenate(axis=1)([nbeats_layer, x])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[time_input, non_time_input], outputs=output)
X_non_time = df[timesteps:, [0, 2, 3, 4]]
model.compile(loss='mse')
print([X_train, X_non_time[:len(X_train)]])
model.fit([X_train, X_non_time[:len(X_train)]], y_train, epochs=7)
model.summary(print_fn=lambda x: sys.stdout.write(x + '\n'), expand_nested=True)
Y = model.predict([X_test, X_non_time[len(X_train):]], verbose=1)
model.evaluate([X_test,X_non_time[len(X_train):]], y_test, verbose=1)

plt.plot(y_test, label='Real')
plt.plot(Y, label='Forecast')
plt.legend()
plt.show()

